In [40]:
with open('15.txt') as f:
    data = f.read().splitlines()

ROW_MAX = 4000000

In [41]:
import networkx as nx
import numpy as np
import re
from tqdm import tqdm
from multiprocessing import Pool

In [42]:
###3
def manhattan_distance(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def return_impossible_beacon_pos(ROW_TO_CHECK, ROW_MAX=ROW_MAX, RETURN_TUNING_FREQ=True):
    set_no_beacons = set()
    sensor_list = []
    beacon_list = []
    for line_id, line in enumerate(data, 1):
        if not RETURN_TUNING_FREQ:
            print(f'Current line: {line_id} / {len(data)}')
        sensor_x, sensor_y, beacon_x, beacon_y = re.findall(r"-?\d+", line)
        # Cast them to int
        sensor_x, sensor_y, beacon_x, beacon_y = int(sensor_x), int(sensor_y), int(beacon_x), int(beacon_y)
        # Add them to the list
        sensor_list.append((sensor_y, sensor_x))
        beacon_list.append((beacon_y, beacon_x))
        
        max_distance = manhattan_distance((sensor_y, sensor_x), (beacon_y, beacon_x))
        y_height_row_to_sensor = abs(sensor_y - ROW_TO_CHECK)
        # Check if the row is between the sensor and the beacon in the y axis
        if abs(sensor_y - ROW_TO_CHECK) <= y_height_row_to_sensor:
            x_distance = max_distance - y_height_row_to_sensor
            for x in range(sensor_x - x_distance, sensor_x + x_distance + 1):
                impossible_beacon = (ROW_TO_CHECK, x)
                if RETURN_TUNING_FREQ:
                    if x <= ROW_MAX and x >= 0:
                        set_no_beacons.add((ROW_TO_CHECK, x))
                    else:
                        pass
                else:
                    set_no_beacons.add((ROW_TO_CHECK, x))
                
    # Remove the beacons or sensors that are in the set
    if not RETURN_TUNING_FREQ:
        for sensor in sensor_list:
            if sensor in set_no_beacons:
                set_no_beacons.remove(sensor)
                
        for beacon in beacon_list:
            if beacon in set_no_beacons:
                set_no_beacons.remove(beacon)
            
    if RETURN_TUNING_FREQ:
        if len(set_no_beacons) <= ROW_MAX:
            # Return missing element of this ROW
            for x in range(ROW_MAX):
                if (ROW_TO_CHECK, x) not in set_no_beacons:
                    result = f'Beacon tuning position: {x*4000000 + ROW_TO_CHECK}'
                    np.savetxt('result_2.txt', np.array([result], dtype=str), fmt='%s')
    else:
        return set_no_beacons


In [51]:
ROW_TO_CHECK = 200000
set_no_beacons = return_impossible_beacon_pos(ROW_TO_CHECK, ROW_MAX, RETURN_TUNING_FREQ=False)

Current line: 1 / 14
Current line: 2 / 14
Current line: 3 / 14
Current line: 4 / 14
Current line: 5 / 14
Current line: 6 / 14
Current line: 7 / 14
Current line: 8 / 14
Current line: 9 / 14
Current line: 10 / 14
Current line: 11 / 14
Current line: 12 / 14
Current line: 13 / 14
Current line: 14 / 14


In [52]:
"""%%timeit
RANGE_ROWS = range(40)
with Pool(20) as p:
    p.map(return_impossible_beacon_pos, RANGE_ROWS)"""

KeyboardInterrupt: 

In [43]:
MAX_ROW = 4000000
import numpy as np
##
sensor_list = []
beacon_list = []
possible_beacon_pos = set()

for line_id, line in enumerate(data, 1):
    print(f'Current line: {line_id} / {len(data)}')
    sensor_x, sensor_y, beacon_x, beacon_y = map(int, re.findall(r"-?\d+", line))
    
    max_distance = manhattan_distance((sensor_y, sensor_x), (beacon_y, beacon_x))
    current_x = sensor_x
    current_y = sensor_y - max_distance - 1
    direction_x = 1
    direction_y = 1
    
    while True:
        if current_y >= 0 and current_y <= MAX_ROW and current_x >= 0 and current_x <= MAX_ROW:
            possible_beacon_pos.add((current_y, current_x))
        current_x += direction_x
        current_y += direction_y
        if sensor_x == current_x and sensor_y - max_distance - 1 == current_y:
            break
        if current_y == sensor_y:
            direction_x *= -1
        if current_x == sensor_x:
            direction_y *= -1
        else:
            pass

    sensor_list.append((sensor_y, sensor_x))
    beacon_list.append((beacon_y, beacon_x))
    
    for (sensor_y, sensor_x), (beacon_y, beacon_x) in zip(sensor_list, beacon_list):
        max_distance = manhattan_distance((sensor_y, sensor_x), (beacon_y, beacon_x))
        to_remove = set()
        for possible_beacon_pos_ in possible_beacon_pos:
            if manhattan_distance((sensor_y, sensor_x), possible_beacon_pos_) <= max_distance:
                to_remove.add(possible_beacon_pos_)
        possible_beacon_pos -= to_remove
        
        

Current line: 1 / 35
Current line: 2 / 35
Current line: 3 / 35
Current line: 4 / 35
Current line: 5 / 35
Current line: 6 / 35
Current line: 7 / 35
Current line: 8 / 35
Current line: 9 / 35
Current line: 10 / 35
Current line: 11 / 35
Current line: 12 / 35
Current line: 13 / 35
Current line: 14 / 35
Current line: 15 / 35
Current line: 16 / 35
Current line: 17 / 35
Current line: 18 / 35
Current line: 19 / 35
Current line: 20 / 35
Current line: 21 / 35
Current line: 22 / 35
Current line: 23 / 35
Current line: 24 / 35
Current line: 25 / 35
Current line: 26 / 35
Current line: 27 / 35
Current line: 28 / 35
Current line: 29 / 35
Current line: 30 / 35
Current line: 31 / 35
Current line: 32 / 35
Current line: 33 / 35
Current line: 34 / 35
Current line: 35 / 35


In [44]:
possible_beacon_pos

{(3249288, 2978645)}

In [50]:
4000000 * list(possible_beacon_pos)[0][1] + list(possible_beacon_pos)[0][0]

11914583249288